In [2]:
# Importing Modules
import getpass # For users
import h5py    # For data files input
import numpy as np
import time

In [3]:
# Setting up data input directory
user = getpass.getuser()
desktop_or_laptop = 'd' # 'l' # This is for Dan

if user == 'scgst':
    with h5py.File('images_training.h5','r') as H:
        train_data = np.copy(H['data'])
    with h5py.File('labels_training.h5','r') as H:
        train_label = np.copy(H['label'])
    with h5py.File('images_testing.h5','r') as H:
        test_data = np.copy(H['data'])
    with h5py.File('labels_testing_2000.h5','r') as H:
        test_label = np.copy(H['label'])  
        
## This is for Dan
# elif user == '': 
#     if desktop_or_laptop == 'l':
#         ## Dan's Mac folder location - NEEDS CHANGING
#         with h5py.File('../Project1/data/images_training.h5','r') as H:
#             data = np.copy(H['data'])
#         with h5py.File('../Project1/data/labels_training.h5','r') as H:
#             label = np.copy(H['label'])
#         with h5py.File('../Project1/data/images_testing.h5','r') as H:
#             data_test = np.copy(H['data'])
#         with h5py.File('../Project1/data/labels_testing_2000.h5','r') as H:
#             label_test = np.copy(H['label'])    
#     else:
#         ## Dan's desktop folder location - NEEDS CHANGING
#         with h5py.File('../../Input/images_training.h5','r') as H:
#             data = np.copy(H['data'])
#         with h5py.File('../../Input/labels_training.h5','r') as H:
#             label = np.copy(H['label'])
#         with h5py.File('../../Input/images_testing.h5','r') as H:
#             data_test = np.copy(H['data'])
#         with h5py.File('../../Input/labels_testing_2000.h5','r') as H:
#             label_test = np.copy(H['label'])

## This is for Ben
# elif user == '':
#     with h5py.File('images_training.h5','r') as H:
#         data = np.copy(H['data'])
#     with h5py.File('labels_training.h5','r') as H:
#         label = np.copy(H['label'])
#     with h5py.File('images_testing.h5','r') as H:
#         data_test = np.copy(H['data'])
#     with h5py.File('labels_testing_2000.h5','r') as H:
#         label_test = np.copy(H['label'])  

In [4]:
# Transforming the data inputs
train_data = train_data.reshape(-1, 784)
test_data = test_data.reshape(-1, 784)

print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

(30000, 784)
(30000,)
(5000, 784)
(2000,)


In [6]:
#### Split Train into train + validation

# Not using sklearn pipeline
from random import sample

index_sample = np.array(sample([i for i in range(1, 30001, 1)], 10000))
data_sample = train_data[index_sample]
label_sample = train_label[index_sample]

from sklearn.model_selection import train_test_split
train_data, validate_data, train_label, validate_label = train_test_split(data_sample, label_sample, test_size = 0.3, random_state=109)

print(train_data.shape)
print(validate_data.shape)
print(train_label.shape)
print(validate_label.shape)



(7000, 784)
(3000, 784)
(7000,)
(3000,)


In [7]:
# Function to reclassify label into only two classes, target class = 1 & everything else = 0
# This is for one to all classification method for SVM
def relabel_two_class(label, target_class):
    label_two_class = np.zeros(len(label))
    class_index = np.where(label == target_class)[0] # non_class_index = np.where(label != target_class)[0]
    label_two_class[class_index] = 1

    return label_two_class

In [9]:
# Fitting SVM model
from sklearn import svm # SVC # "Support vector classifier"

start_0 = time.time()

# Classify 0
label_class_0 = relabel_two_class(train_label, 0)
model_0 = svm.SVC(kernel = 'rbf', C = 1E6) # kernel could also be linear, etc..
model_0.fit(train_data, label_class_0)

end_0 = time.time()
print(end_0 - start_0)

C:\Users\scgst\Documents\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


62.56083035469055


In [ ]:
# Prediction
label_pred = model_0.predict(validate_data)

In [7]:
# Evaluating performance
from sklearn import metrics

label_test = relabel_two_class(validate_label, 0)

print(metrics.accuracy_score(label_test, label_pred))
print(metrics.precision_score(label_test, label_pred))
print(metrics.recall_score(label_test, label_pred))

(588, 784)